In [ ]:
# Instalamos la libreria
!pip install oci pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 45.2 MB/s eta 0:00:00


In [ ]:
# Importamos las librerias necesarias
import oci
import pandas as pd
import csv

In [ ]:
from sklearn.preprocessing import StandardScaler

#Tablesarticles.csv

In [ ]:
# DEFINICIÓN DE CONSTANTES
NAMESPACE = 'ax5grcoay8ni'
BUCKET_NAME = 'Team3'
OBJECT_NAME = 'Tablesarticles.csv'
LOCAL_CSV_PATH = 'articles_local.csv'
FINAL_CSV_NAME = 'articles_modified.csv' # Archivo de salida

# URLs Base de los tres buckets
BUCKET_1_URL = 'https://objectstorage.us-chicago-1.oraclecloud.com/n/ax5grcoay8ni/b/Team3/o/' # Bucket con los csv y 10-35
BUCKET_2_URL = 'https://objectstorage.us-chicago-1.oraclecloud.com/n/axym8lqm5eyc/b/bucketia/o/' # Bucket 36-70
BUCKET_3_URL = 'https://objectstorage.us-chicago-1.oraclecloud.com/n/axheitjen1w2/b/reto/o/' # Bucket 71-95

In [ ]:
# Cargamos la configuración
try:
    config = oci.config.from_file("config", "DEFAULT")
    object_storage_client = oci.object_storage.ObjectStorageClient(config)
except Exception as e:
    print(f"Error al cargar la configuración: {e}")

Error al cargar la configuración: Could not find config file at config, please follow the instructions in the link to setup the config file https://docs.cloud.oracle.com/en-us/iaas/Content/API/Concepts/sdkconfig.htm


In [ ]:
# Descargamos el objeto CSV
get_object_response = object_storage_client.get_object(NAMESPACE, BUCKET_NAME, OBJECT_NAME)

with open(LOCAL_CSV_PATH, 'wb') as f:
    for chunk in get_object_response.data.raw.stream(1024 * 1024, decode_content=False):
        f.write(chunk)

df_articles = pd.read_csv(LOCAL_CSV_PATH)

print(f"'{OBJECT_NAME}' descargado exitosamente. Filas: {len(df_articles)}")
print("-" * 50)

# Visualizamos
print("Vista preliminar del DataFrame original:")
df_articles.head()

NameError: name 'object_storage_client' is not defined

In [ ]:
# Esto es para ver todos los caracteres de la url
pd.set_option('display.max_colwidth', 200)

def generate_image_url(article_id):
    # Convierte el article_id a string para la manipulación
    article_id_str_full_jpg = str(article_id).zfill(10)

    # Extraemos los primeros dos dígitos del article_id del CSV
    prefix_2_digits_from_csv = str(article_id)[0:2]

    # Convertimos a 3 dígitos para el nombre de la carpeta (ej: '011' de '11')
    folder_name_3_digits = prefix_2_digits_from_csv.zfill(3)

    # Lógica de asignación de Bucket
    if '10' <= prefix_2_digits_from_csv <= '35':
        base_url = BUCKET_1_URL

    elif '36' <= prefix_2_digits_from_csv <= '70':
        base_url = BUCKET_2_URL

    elif '71' <= prefix_2_digits_from_csv <= '95':
        base_url = BUCKET_3_URL

    else:
        return None

    # La clave final del objeto es: images/011/0110065002.jpg
    object_key = f"images/{folder_name_3_digits}/{article_id_str_full_jpg}.jpg"

    return base_url + object_key

# Aplicamos la función para crear la nueva columna
df_articles[f'img_url_{BUCKET_NAME}'] = df_articles['article_id'].apply(generate_image_url)

print("Columna de URLs de imagen generada.")

In [ ]:
# Visualizamos el article_id junto con su imagen asociada
df_articles[['article_id', f'img_url_{BUCKET_NAME}']].head(90000)

In [ ]:
df_articles.head()

**Transformacion**

In [ ]:
df_articles.info()

In [ ]:
df_articles.isnull().sum()

In [ ]:
df_articles.describe()

In [ ]:
# Analiza todas las columnas y muestra la cuenta de valores únicos
df_articles.nunique()

In [ ]:
# Analiza la distribución de una columna categórica clave (ej: product_group_name)
df_articles['product_group_name'].value_counts()

In [ ]:
# O de una columna textual/nula (ej: garment_group_name)
df_articles['garment_group_name'].value_counts(dropna=False)

**Imputacion de valores nulos**

La columna 'detail_desc' tiene 416 valores nulos.

In [ ]:
df_articles['detail_desc'] = df_articles['detail_desc'].fillna('sin descripcion detallada')

**Ingeniería de Características para RAG (Retrieval-Augmented Generation)**

 Creamos una columna de texto única que consolida las características más relevantes del producto.
 Este será el 'chunk' de texto que OCI Generative AI/Embeddings vectorizará para la búsqueda semántica.

In [ ]:
df_articles['description_vector_rag'] = (
    "NOMBRE: " + df_articles['prod_name'] +
    "; GRUPO: " + df_articles['product_group_name'] +
    "; COLOR: " + df_articles['colour_group_name'] +
    "; TIPO: " + df_articles['product_type_name'] +
    "; DETALLE: " + df_articles['detail_desc']
)

**Definición de Columnas para one hot y Eliminación**

Columnas categóricas de baja cardinalidad (útiles para el modelo de recomendación)

In [ ]:
OHE_cols_name = [
    'product_type_name', 'product_group_name', 'graphical_appearance_name',
    'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name',
    'index_name', 'index_group_name', 'section_name', 'garment_group_name'
]
df_articles = pd.get_dummies(df_articles, columns=OHE_cols_name)

**Convertimos de booleanos a binarios**

In [ ]:
ID_and_Redundant_cols = [
    'product_code', 'prod_name', 'detail_desc', 'department_name', # prod_name y detail_desc deben ir aquí
    'product_type_no', 'graphical_appearance_no', 'colour_group_code',
    'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no',
    'index_code', 'index_group_no', 'section_no', 'garment_group_no'
]

# Aseguramos que la eliminación se haga antes de la conversión de tipos
df_articles.drop(columns=ID_and_Redundant_cols, inplace=True, errors='ignore')
print("Columnas redundantes y de texto original eliminadas.")

In [ ]:
text_and_id_cols = ['article_id', f'img_url_{BUCKET_NAME}', 'description_vector_rag']
ohe_cols_to_convert = [col for col in df_articles.columns if col not in text_and_id_cols]

for col in ohe_cols_to_convert:
    # Forzamos la conversión a int (maneja True/False)
    df_articles[col] = df_articles[col].astype(int)
    # Convertimos a string ('1' o '0') para evitar que Pandas escriba True/False en el CSV
    df_articles[col] = df_articles[col].astype(str)

print("Conversión final de OHE a strings '1' y '0' completada.")

**Eliminación de Columnas de Identificación y Texto Original**

Eliminamos las columnas originales que han sido reemplazadas por OHE o son de alta cardinalidad/ruido.

In [ ]:
ID_and_Redundant_cols = [
    'product_code', 'prod_name', 'detail_desc', 'department_name',
    'product_type_no', 'graphical_appearance_no', 'colour_group_code',
    'perceived_colour_value_id', 'perceived_colour_master_id', 'department_no',
    'index_code', 'index_group_no', 'section_no', 'garment_group_no'
]

df_articles.drop(columns=ID_and_Redundant_cols, inplace=True, errors='ignore')

**Cambios realizados**

In [ ]:
print(f" Columnas totales: {df_articles.shape[1]}")
print(df_articles.head())

In [ ]:
print(f"Filas totales: {df_articles.shape[0]}")
print(f"Columnas totales: {df_articles.shape[1]}")

In [ ]:
# def generate_sql_create_table(df):
#     """Genera la sentencia CREATE TABLE con tipos de datos correctos."""
#     sql_columns = []

#     # Mapeo de columnas principales de texto/ID
#     # Usaremos 4000 bytes para los VARCHAR2 por seguridad.
#     for col in df.columns:
#         if col == 'article_id':
#             sql_columns.append(f'"{col}" NUMBER')
#         elif col.startswith('img_url_') or col == 'description_vector_rag':
#             # Mantenemos VARCHAR2 para las columnas RAG y URL
#             sql_columns.append(f'"{col}" VARCHAR2(4000)')
#         else:
#             # Todas las OHE deben ser NUMBER (solo contienen '1' o '0')
#             sql_columns.append(f'"{col}" NUMBER')

#     # Unir todas las definiciones de columna
#     columns_definition = ",\n".join(sql_columns)

#     # Crear la sentencia CREATE TABLE final
#     create_table_sql = (
#         f"CREATE TABLE ARTICLES_MODIFIED (\n{columns_definition}\n);"
#     )
#     return create_table_sql

# # Generar y mostrar la sentencia SQL
# sql_statement = generate_sql_create_table(df_articles)
# print(sql_statement)

**Guardamos y subimos los cambios en el archivo de OCI**

In [ ]:
# df_articles.to_csv(
#     FINAL_CSV_NAME,
#     index=False,
#     encoding="utf-8",
#     quoting=csv.QUOTE_ALL  # <-- cada campo irá entre comillas
# )

# try:
#     with open(FINAL_CSV_NAME, 'rb') as f:
#         # Sobreescribir el archivo final en el bucket OCI
#         object_storage_client.put_object(
#             NAMESPACE,
#             BUCKET_NAME,
#             FINAL_CSV_NAME,
#             f
#         )
#     print(f"\n '{FINAL_CSV_NAME}' (Artículos transformados) SUBIDO exitosamente al bucket.")
# except Exception as e:
#     print(f"\n Error al subir el archivo: {e}")

#Tablescustomers.csv

In [ ]:
# Nuestras constantes
CUST_OBJECT_NAME = 'Tablescustomers.csv'
CUST_LOCAL_PATH = 'customers_local.csv'
CUST_FINAL_NAME = 'customers_modified.csv'

In [ ]:
# Descargamos el csv que tenemos en el bucket y lo cargamos aqui para manipularlo con pandas
get_cust_response = object_storage_client.get_object(NAMESPACE, BUCKET_NAME, CUST_OBJECT_NAME)
with open(CUST_LOCAL_PATH, 'wb') as f:
    for chunk in get_cust_response.data.raw.stream(1024 * 1024, decode_content=False):
        f.write(chunk)
df_customers = pd.read_csv(CUST_LOCAL_PATH)

In [ ]:
df_customers.head()

**Transformacion**

In [ ]:
df_customers.info()

In [ ]:
df_customers.isnull().sum()

In [ ]:
df_customers.describe()

In [ ]:
df_customers.nunique()

**Eliminamos duplicados**

In [ ]:
df_customers.drop_duplicates(subset=['customer_id'], keep='first', inplace=True)

**Imputación Binaria y Conversión de Flags**

Columnas 'FN' y 'Active' son flags binarios (1.0 o NaN). Imputamos NaN con 0.

In [ ]:
for col in ['FN', 'Active']:
    # Imputamos los nulos con 0.0 (asumiendo que es un valor "No" o "Desconocido")
    df_customers[col] = df_customers[col].fillna(0.0)
    # Convertimos a entero para asegurar un tipo de dato binario limpio
    df_customers[col] = df_customers[col].astype(int)

**Imputación y Estandarización de la Edad**

Calculamos la mediana de la edad para una imputación robusta

In [ ]:
median_age = df_customers['age'].median()

# Reasignamos el resultado de fillna a la columna 'age'
df_customers['age'] = df_customers['age'].fillna(median_age)

# Aplicamos la Estandarización (StandardScaler)
scaler = StandardScaler()
df_customers['age_scaled'] = scaler.fit_transform(df_customers[['age']])

**Imputación y One-Hot Encoding (OHE) de Categorías**

In [ ]:
# Columnas categóricas a transformar
OHE_cols_cust = ['club_member_status', 'fashion_news_frequency']

# Imputación Categórica: Rellenamos los nulos con 'UNKNOWN'
for col in OHE_cols_cust:
    df_customers[col] = df_customers[col].fillna('UNKNOWN')

print("Imputación de club_member_status y fashion_news_frequency completada.")

# Aplicamos One-Hot Encoding a las columnas categóricas
df_customers = pd.get_dummies(df_customers, columns=OHE_cols_cust, prefix=OHE_cols_cust)

**Hacemos la conversion binaria**

In [ ]:
cols_to_convert_to_int = [col for col in df_customers.columns if df_customers[col].dtype == 'bool']

for col in cols_to_convert_to_int:
    # Convertir el tipo de dato 'bool' a entero 'int' (True -> 1, False -> 0)
    df_customers[col] = df_customers[col].astype(int)

**Eliminación de Columnas Finales**

In [ ]:
columns_to_drop_cust = ['age', 'postal_code', 'club_member_status', 'fashion_news_frequency']
df_customers.drop(columns=columns_to_drop_cust, inplace=True, errors='ignore')

**Previsualizacion**

In [ ]:
print(f" Columnas totales: {df_customers.shape[1]}")
print(df_customers.head())

**Guardamos y subimos los cambios en el archivo a OCI**

In [ ]:
# df_customers.to_csv(CUST_FINAL_NAME, index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)

# try:
#     with open(CUST_FINAL_NAME, 'rb') as f:
#         # Sobreescribir el archivo final en el bucket OCI
#         object_storage_client.put_object(
#             NAMESPACE,
#             BUCKET_NAME,
#             CUST_FINAL_NAME,
#             f
#         )
#     print(f"\n '{CUST_FINAL_NAME}' (Clientes transformados) subido exitosamente al bucket '{BUCKET_NAME}'.")
# except Exception as e:
#     print(f"\n Error al subir el archivo: {e}")

In [ ]:
df_customers.shape[0]

In [ ]:
# def generate_sql_create_table_customers(df_customers):
#     """Genera la sentencia CREATE TABLE para la tabla de Clientes (CUSTOMERS_MODIFIED)."""

#     sql_columns = []

#     # Iterar sobre las columnas del DataFrame de Clientes
#     for col, dtype in df_customers.dtypes.items():

#         # 1. Definición de la Clave Primaria (customer_id)
#         if col == 'customer_id':
#             # customer_id es un hash (string largo), no un número.
#             sql_columns.append(f'"{col}" VARCHAR2(128)')

#         # 2. Definición de la Edad Escalada, FN, Active, y OHEs
#         elif col in ['FN', 'Active', 'age_scaled'] or str(dtype) in ['int64', 'float64']:
#             # Todos los flags OHE, FN, Active, y la edad escalada son números
#             sql_columns.append(f'"{col}" NUMBER')

#         else:
#             # Columna de respaldo (VARCHAR2 por defecto)
#             sql_columns.append(f'"{col}" VARCHAR2(4000)')

#     # Unir todas las definiciones de columna
#     columns_definition = ",\n".join(sql_columns)

#     # Crear la sentencia CREATE TABLE final
#     create_table_sql = (
#         f"DROP TABLE CUSTOMERS_MODIFIED;\n\n"
#         f"CREATE TABLE CUSTOMERS_MODIFIED (\n{columns_definition}\n);"
#     )
#     return create_table_sql

# # --- Ejecución ---
# sql_statement = generate_sql_create_table_customers(df_customers)
# print(sql_statement)

#Tablestransactions_train.csv

In [ ]:
# Nuestras constantes
TRANS_OBJECT_NAME = 'Tablestransactions_train.csv'
TRANS_LOCAL_PATH = 'transactions_local.csv'
TRANS_FINAL_NAME = 'transactions_modified.csv'

In [ ]:
# Descargamos el csv que tenemos en el bucket y lo cargamos aqui para manipularlo con pandas

# Ejecutamos la llamada a la API de OCI para obtener el objeto.
get_trans_response = object_storage_client.get_object(NAMESPACE, BUCKET_NAME, TRANS_OBJECT_NAME)

# Guardamos la respuesta de OCI en un archivo local
with open(TRANS_LOCAL_PATH, 'wb') as f:
    for chunk in get_trans_response.data.raw.stream(1024 * 1024, decode_content=False):
        f.write(chunk)

# Leemos el archivo local con pandas
df_transactions = pd.read_csv(TRANS_LOCAL_PATH)

In [ ]:
df_transactions.head()

**Transformacion**

In [ ]:
df_transactions.info()

In [ ]:
df_transactions.isnull().sum()

In [ ]:
df_transactions.describe()

In [ ]:
df_transactions.nunique()

**Ingeniería de Características de Tiempo (t_dat)**

La columna 't_dat' fue cargada como 'object' (string). Convertimos a datetime.

In [ ]:
# Convertimos 't_dat' a datetime (si no se hizo en la carga)
df_transactions['t_dat'] = pd.to_datetime(df_transactions['t_dat'])

# Extraemos features temporales clave
df_transactions['t_year'] = df_transactions['t_dat'].dt.year
df_transactions['t_month'] = df_transactions['t_dat'].dt.month
df_transactions['t_dayofweek'] = df_transactions['t_dat'].dt.dayofweek
df_transactions['t_day'] = df_transactions['t_dat'].dt.day
print("Características de tiempo extraídas.")

**Escalamiento de 'price'**

Estandarizamos el precio para que su magnitud no domine el modelo de recomendación.

In [ ]:
scaler = StandardScaler()
df_transactions['price_scaled'] = scaler.fit_transform(df_transactions[['price']])
print("Columna 'price' escalada a 'price_scaled'.")

**One-Hot Encoding (OHE) de Canales de Venta**

Aplicamos OHE a 'sales_channel_id' (que tiene solo 2 valores únicos: 1 y 2)

In [ ]:
df_transactions = pd.get_dummies(
    df_transactions,
    columns=['sales_channel_id'],
    prefix='channel',
    drop_first=True # Opcional: Elimina una columna para evitar multicolinealidad
)
print("One-Hot Encoding aplicado a 'sales_channel_id'.")

**Pasamos los booleanos a binarios**

In [ ]:
df_transactions['channel_2'] = df_transactions['channel_2'].astype(int)
df_transactions['channel_2'] = df_transactions['channel_2'].astype(str)

**Eliminación de Columnas Finales y Limpieza**

Eliminamos las columnas originales que han sido reemplazadas o ya no son necesarias.

In [ ]:
df_transactions.drop(
    columns=['t_dat'],
    inplace=True,
    errors='ignore'
)
print("Columna 't_dat' original eliminada.")

**Previsualizacion final**

In [ ]:
print(f" Columnas totales: {df_transactions.shape[1]}")
print(df_transactions.head())

In [ ]:
df_transactions.shape[0]

**Guardamos y subimos los cambios en el archivo a OCI**

In [ ]:
# df_transactions.to_csv(TRANS_FINAL_NAME, index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)

# try:
#     with open(TRANS_FINAL_NAME, 'rb') as f:
#         # Sobreescribir el archivo final en el bucket OCI
#         object_storage_client.put_object(
#             NAMESPACE,
#             BUCKET_NAME,
#             TRANS_FINAL_NAME,
#             f
#         )
#     print(f"\n '{TRANS_FINAL_NAME}' (Transacciones transformadas) subido exitosamente al bucket '{BUCKET_NAME}'.")
# except Exception as e:
#     print(f"\n Error al subir el archivo: {e}")